# Generate automated session reports for all participants

In [ ]:
import pandas as pd
import numpy as np
import mne
import mne_bids
import glob
import warnings
import os
import re
import subprocess

import q1k_pyll_tools as qpt

warnings.filterwarnings('ignore')

### Select task parameters and file paths

In [ ]:
# Select kind of data experimental or control group

#dataset_group = "experimental"

#if dataset_group == "control":
#    # Control group data
#    project_path = "/home/james/q1k/pilot/q1k-external-pilot/"
#    task_id_in = "ap"
#    task_id_in_et = "ap"
#    task_id_out = "ap"
#    #subject_id = '002'
#    session_id = '01'
#    run_id = '1'

#elif dataset_group == "experimental":
## Experimental group data

project_path = "/project/def-emayada/q1k/experimental/HSJ/"
init_path = "code/q1k_eeget_init/"
task_id_in = "VEP"
task_id_in_et = "VEP"
task_id_out = "VEP"
#subject_id = 'Q1K_HSJ_100123_F1'
run_id = '1'
session_id = '01'
root_data_path = "" 
derivative_path = "derivatives/pylossless/"



In [ ]:
#find existing *.edf files 
pattern = project_path + derivative_path + '**/eeg/*' + task_id_out + '*.edf'
processed_sessions = glob.glob(pattern, recursive=True)
        
pattern = project_path + '**/eeg/*' + task_id_in + '*.edf'
file_paths = glob.glob(pattern, recursive=True)
for file_path in file_paths:
    file_name = os.path.basename(file_path)  # Extract the filename from the full path
    print(f"File Path: {file_path}, File Name: {file_name}")
    
    # Skip sessions that have already been processed
    if file_path in processed_sessions:
        print(file_name + ' has already been processed')
        continue    

    sbatch_command, result = qpt.submit_slurm_job(project_path, file_name)